**Broadcasting** is a powerful mechanism that allows numpy to work with arrays of different shapes when performing arithmetic operations. Frequently we have a smaller array and a larger array, and we want to use the smaller array multiple times to perform some operation on the larger array.
For example, suppose that we want to add a constant vector to each row of a matrix. We could do it like this:

In [5]:
import numpy as np

# We will add the vector v to each row of the matrix x,
# storing the result in the matrix y
x = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
v = np.array([1, 0, 1])

# Return a new array with the same shape and type as a given array
# This function does not initialize the returned array
y = np.empty_like(x)
print(y)

# Add the vector v to each row of the matrix x with an explicit loop
for i in range(4):
    y[i, :] = x[i, :] + v
    
print(y)

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]
[[ 2  2  4]
 [ 5  5  7]
 [ 8  8 10]
 [11 11 13]]


This works; however when the matrix x is very large, computing an explicit loop in Python could be slow. Note that adding the vector v to each row of the matrix x is equivalent of forming a matrix vv by stacking multiple copies of v vectically, then perform elementwise summation of x and vv. We could implement this approach like this:

In [6]:
import numpy as np

# We will add the vector v to each row of the matrix x,
# storing the result in the matrix y
x = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
v = np.array([1, 0, 1])
vv = np.tile(v, (4, 1))    # Stack 4 copies of v on top of each other
print(vv)

y = x + vv  # Add x and vv elementwise
print(y)

[[1 0 1]
 [1 0 1]
 [1 0 1]
 [1 0 1]]
[[ 2  2  4]
 [ 5  5  7]
 [ 8  8 10]
 [11 11 13]]


**Numpy broadcasting** allows us to perform this computation without actually creating multiple copies of v. Consider this version, using **broadcasting**:

In [10]:
import numpy as np

# We will add the vector v to each row of the matrix x,
# storing the result in the matrix y
x = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
print("x shape: ",x.shape)
v = np.array([1, 0, 1])
print("v shape: ",v.shape)
y = x + v  # Add v to each row of x using broadcasting
print(y)

x shape:  (4, 3)
v shape:  (3,)
[[ 2  2  4]
 [ 5  5  7]
 [ 8  8 10]
 [11 11 13]]


The line _y_ = _x_ + _v_ works even though _x_ has shape (4, 3) and _v_ has shape(3,) due to broadcasting; this line works as if _v_ actually had shape (4, 3), where each row was a copy of _v_, and the sum was performed elementwise.

**Broadcasting** two arrays together follows these rules:
1. If the arrays do not have the same rank, pretend the shape of the lower rank array with 1s until both shapes have the the same length,
2. The two arrays are said to be _compatible_ in a dimension if they have the same size in the dimension, or if one of the arrays has size 1 in that dimension.
3. The array can be broadcast together if they are _compatible_ in all dimensions.
4. After broadcasting, each array behaves as if it had shape equal to the elementwise maximum of shapes of the two input arrays.
5. In any dimension where one array has size 1 and the other array had size greater than 1, the first array behaves as if it were copied along that dimension.

Functions that support **broadcasting** are known as _universal functions_. **Broadcasting** typically makes your code more concise and faster, so you should strive to ust it where possible. Here are some applications of broadcasting:

In [14]:
import numpy as np

# Compute outer product of vectors
v = np.array([1, 2, 3])
print("v has shape: ",v.shape)
w = np.array([4, 5])
print("w has shape: ",w.shape)

# To compute an outer product, we first reshape v to be a column vector of shape (3, 1);
# We can then broadcast it against w to yield an output of shape (3, 2), which is the outer
# product of v and w;
# v : [[1],    ->     [[1, 1],
#      [2],            [2, 2],
#      [3]]            [3, 3]]
# w : [4, 5]   ->     [[4, 5],
#                      [4, 5],
#                      [4, 5]]
# Finally, the reshape matrices multiply elementwise, then get the outer product.
print(np.reshape(v, (3, 1)) * w)
print("\n")

# Add a vector to each column of a matrix
# x has shape (2, 3) and w has shape (2,).
# If we transpose x then it has shape (3, 2) and can be broadcast against w to yield a result
# of shape (3, 2); transposing this result yields the final result of shape (2, 3) which is the
# matrix x with the vector w added to each column. Give the following matrix:
x = np.array([[1, 2, 3], [4, 5, 6]])
print((x.T + w).T)
print("\n")

# Another solution is to reshape w to be a column vector of shape (2, 1);
# we can then broadcast it directly against x to produce the same output.
print(x + np.reshape(w, (2, 1)))
print("\n")

# Multiply a matrix by a constant:
# x has shape (2, 3). Numpy treats scalars as arrays of shape ();
# these can be broadcast together to shape (2, 3), producing the
# following array:
print(x * 2)

v has shape:  (3,)
w has shape:  (2,)
[[ 4  5]
 [ 8 10]
 [12 15]]


[[ 5  6  7]
 [ 9 10 11]]


[[ 5  6  7]
 [ 9 10 11]]


[[ 2  4  6]
 [ 8 10 12]]


## Image operations
Numpy provides a high-performance multidimensional array and basic tools to compute with and manipulate these arrays. SciPy builds on this, and provides a large number of functions that operate on numpy arrays and are useful for different types of scientific and engineering applications.
SciPy provides some basic functions to work with images. For example, it has functions to read images from disk into numpy arrays, to write numpy arrays to disk as images, and to resize images. Here is a simple example that showcases these functions:

In [16]:
from scipy.misc import imread, imsave, imresize

# Read an JPEG image into a numpy array
img = imread('cat.jpg')
print(img.dtype, img.shape)

# We can tint the image by scaling each of the color channels by a different scalar constant.
# The image has shape (400, 248, 3); we multiply it by the array [1, 0.95, 0.9] of shape (3,);
# numpy broadcasting means that this leaves the read channel unchanged, and multiplies the
# green and blue channels by 0.95 and 0.9 respectively.
img_tinted = img * [1, 0.95, 0.9]

# Resize the tinted image to be 300 by 300 pixels.
img_tinted = imresize(img_tinted, (300, 300))

# Write the tinted image back to disk
imsave('cat_tinted.jpg', img_tinted)

uint8 (400, 248, 3)
